In [4]:
import sys
print(sys.path)
import torch

['/Users/ben/Desktop/DISSERTATION/github/dissertation2022', '/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python39.zip', '/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9', '/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/lib-dynload', '', '/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow/lib/python3.9/site-packages']


ModuleNotFoundError: No module named 'torch'

In [5]:

import pandas as pd
df = pd.read_csv('/Users/ben/Desktop/DISSERTATION/github/dissertation2022/kagglenewsdataset.csv')
df.columns.tolist()


['Unnamed: 0',
 'category',
 'headline',
 'authors',
 'link',
 'short descr',
 'date',
 'image_caption(s)',
 'image_link(s)']

In [2]:
shortdf = df[1:100]

In [3]:
shortdf

,Unnamed: 0,category,headline,authors,link,short descr,date,image_caption(s),image_link(s)
1,1,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,"Donna Bloom, director of legal services at Den...",https://img.huffingtonpost.com/asset/5b081d6c1...
2,2,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/5b081c9f1...
3,3,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/5b0844b51...
4,4,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,"Melissa Jeltsen - Senior Reporter, HuffPost",https://img.huffingtonpost.com/asset/5c1128ec2...
5,5,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/62bb083f2...
...,...,...,...,...,...,...,...,...,...
95,95,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,Todd Van Luling,https://www.huffingtonpost.com/entry/hulu-what...,You're getting a recent Academy Award-winning ...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/62b7ebfc2...
96,96,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,Todd Van Luling,https://www.huffingtonpost.com/entry/hulu-what...,You're getting a recent Academy Award-winning ...,2018-05-26,"Todd Van Luling - Senior Culture Reporter, Huf...",https://img.huffingtonpost.com/asset/5e5b285f2...
97,97,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,Todd Van Luling,https://www.huffingtonpost.com/entry/hulu-what...,You're getting a recent Academy Award-winning ...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/62bb3d1d2...
98,98,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,Todd Van Luling,https://www.huffingtonpost.com/entry/hulu-what...,You're getting a recent Academy Award-winning ...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/62bae6ed2...


In [8]:
!pip install clip-client

In [6]:
from clip_client import Client
from docarray import Document
df2=[]
scores = []
c = Client(server='grpc://0.0.0.0:51000')
for i, row in shortdf.iterrows():
    r = c.rank(
        [
            Document(
                uri=row['image_link(s)'],
                matches=[
                    Document(text=f'{p}')
                    for p in (
                        row['headline'],row['short descr'],row['image_caption(s)']
                    )
                ],
            )
        ]
    )
    col=[]

#     print(r['@m', ['text', 'scores__clip_score__value']])
    print(r['@m', ['text']])
    print(r['@m', ['scores__clip_score__value']])
    score = [0,0,0]
    for i in range(0,len(r), 3):
        for j in range(3):
            col.append(r['@m', ['text']][i+j])
            if (r['@m', ['text']][i+j] == row['headline']):
                print('inside here')
                score[0] = r['@m', ['scores__clip_score__value']][i+j]
            if (r['@m', ['text']][i+j] == row['short descr']):
                score[1] = r['@m', ['scores__clip_score__value']][i+j]
            if (r['@m', ['text']][i+j] == row['image_caption(s)']):
                score[2] = r['@m', ['scores__clip_score__value']][i+j]
            
            

            col.append(r['@m', ['scores__clip_score__value']][i+j])
        scores.append(score)
        df2.append(col)
df3 = pd.DataFrame(scores, columns = ['headline','short descr','image caption'])

ModuleNotFoundError: No module named 'clip_client'

In [9]:
dff2 = pd.DataFrame(df2)

In [10]:
dff2

,0,1,2,3,4,5
0,"a photo of Donna Bloom, director of legal serv...",0.996866,a photo of There Were 2 Mass Shootings In Texa...,0.002353,a photo of She left her husband. He killed the...,0.000781
1,a photo of There Were 2 Mass Shootings In Texa...,0.851511,a photo of She left her husband. He killed the...,0.148194,a photo of nan,0.000295
2,a photo of There Were 2 Mass Shootings In Texa...,0.970399,a photo of She left her husband. He killed the...,0.029321,a photo of nan,0.000279
3,"a photo of Melissa Jeltsen - Senior Reporter, ...",0.996708,a photo of She left her husband. He killed the...,0.002904,a photo of There Were 2 Mass Shootings In Texa...,0.000388
4,a photo of She left her husband. He killed the...,0.989486,a photo of nan,0.009067,a photo of There Were 2 Mass Shootings In Texa...,0.001447
...,...,...,...,...,...,...
94,a photo of nan,0.621906,a photo of What To Watch On Hulu That’s New Th...,0.305682,a photo of You're getting a recent Academy Awa...,0.072412
95,a photo of Todd Van Luling - Senior Culture Re...,0.998563,a photo of What To Watch On Hulu That’s New Th...,0.001166,a photo of You're getting a recent Academy Awa...,0.000271
96,a photo of What To Watch On Hulu That’s New Th...,0.602507,a photo of nan,0.276361,a photo of You're getting a recent Academy Awa...,0.121132
97,a photo of What To Watch On Hulu That’s New Th...,0.631781,a photo of You're getting a recent Academy Awa...,0.212671,a photo of nan,0.155548


In [11]:
shortdf

,Unnamed: 0,category,headline,authors,link,short descr,date,image_caption(s),image_link(s)
1,1,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,"Donna Bloom, director of legal services at Den...",https://img.huffingtonpost.com/asset/5b081d6c1...
2,2,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/5b081c9f1...
3,3,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/5b0844b51...
4,4,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,"Melissa Jeltsen - Senior Reporter, HuffPost",https://img.huffingtonpost.com/asset/5c1128ec2...
5,5,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/62bb083f2...
...,...,...,...,...,...,...,...,...,...
95,95,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,Todd Van Luling,https://www.huffingtonpost.com/entry/hulu-what...,You're getting a recent Academy Award-winning ...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/62b7ebfc2...
96,96,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,Todd Van Luling,https://www.huffingtonpost.com/entry/hulu-what...,You're getting a recent Academy Award-winning ...,2018-05-26,"Todd Van Luling - Senior Culture Reporter, Huf...",https://img.huffingtonpost.com/asset/5e5b285f2...
97,97,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,Todd Van Luling,https://www.huffingtonpost.com/entry/hulu-what...,You're getting a recent Academy Award-winning ...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/62bb3d1d2...
98,98,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,Todd Van Luling,https://www.huffingtonpost.com/entry/hulu-what...,You're getting a recent Academy Award-winning ...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/62bae6ed2...


In [15]:
df3

,headline,short descr,image caption
0,0.002701,0.002535,0.994763
1,0.878868,0.121125,0.000000
2,0.871205,0.128667,0.000000
3,0.000498,0.002167,0.997335
4,0.013610,0.981711,0.000000
...,...,...,...
94,0.353429,0.143940,0.000000
95,0.000196,0.000439,0.999364
96,0.434632,0.401377,0.000000
97,0.588227,0.289185,0.000000


In [19]:
both_joined = pd.concat([shortdf, df3], axis=1)

In [20]:
both_joined

,Unnamed: 0,category,headline,authors,link,short descr,date,image_caption(s),image_link(s),headline,short descr,image caption
1,1.0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,"Donna Bloom, director of legal services at Den...",https://img.huffingtonpost.com/asset/5b081d6c1...,0.878868,0.121125,0.000000
2,2.0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/5b081c9f1...,0.871205,0.128667,0.000000
3,3.0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/5b0844b51...,0.000498,0.002167,0.997335
4,4.0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,"Melissa Jeltsen - Senior Reporter, HuffPost",https://img.huffingtonpost.com/asset/5c1128ec2...,0.013610,0.981711,0.000000
5,5.0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/62bb083f2...,0.004723,0.971597,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
96,96.0,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,Todd Van Luling,https://www.huffingtonpost.com/entry/hulu-what...,You're getting a recent Academy Award-winning ...,2018-05-26,"Todd Van Luling - Senior Culture Reporter, Huf...",https://img.huffingtonpost.com/asset/5e5b285f2...,0.434632,0.401377,0.000000
97,97.0,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,Todd Van Luling,https://www.huffingtonpost.com/entry/hulu-what...,You're getting a recent Academy Award-winning ...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/62bb3d1d2...,0.588227,0.289185,0.000000
98,98.0,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,Todd Van Luling,https://www.huffingtonpost.com/entry/hulu-what...,You're getting a recent Academy Award-winning ...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/62bae6ed2...,0.275233,0.204012,0.000000
99,99.0,ENTERTAINMENT,What To Watch On Hulu That’s New This Week,Todd Van Luling,https://www.huffingtonpost.com/entry/hulu-what...,You're getting a recent Academy Award-winning ...,2018-05-26,NaN,https://img.huffingtonpost.com/asset/62bad7102...,NaN,NaN,NaN


In [21]:
both_joined.to_csv('checking_scores.csv')